In [1]:
import pandas as pd
from psycopg2 import connect, extensions
from sqlalchemy import create_engine

In [12]:
user = 'airflow'
pw = 'airflow'
ip = '172.19.0.1:5432'
dbname = 'airflow_db'
conn_string = f'postgresql://{user}:{pw}@{ip}/{dbname}'
db = create_engine(conn_string)
conn = db.connect()

In [3]:
dat = pd.read_csv("./test.csv")

In [4]:
dat

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [6]:
table_name = "test_table"
dat.to_sql(name=table_name, con=conn, if_exists='replace', index=False) 
conn.commit()
conn.close()

In [63]:
conn = connect(database="airflow_db",
               host='172.19.0.1',
               port=5432,
               user='airflow',
               password='airflow')
cursor = conn.cursor()

In [64]:
table_name = "test_table"
cursor.execute(f"SELECT * FROM {table_name}")
df = pd.DataFrame(data=cursor.fetchall(),
                  columns=[x.name for x in cursor.description])
cursor.close()
conn.close()

In [65]:
df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,None,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,None,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,None,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,None,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,None,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,None,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,None,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,None,S


In [66]:
import pandas as pd
import requests
import io

In [93]:
import requests
import json

# 데이터프레임을 딕셔너리 리스트로 변환
df = df.fillna('Nan')
data_dict = df.to_dict('records')

# API 요청 데이터 준비
req_data = {
    "data": data_dict
}

# data = req_data['data']
# data = pd.DataFrame(data)

# prediction = model.predict(data)
# prediction.to_dict('records')
# prediction

out = requests.post('http://172.19.0.1:8089/predict', json=req_data)
out

<Response [500]>

In [78]:
pd.DataFrame(req_data['data'])

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Nan,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0,Nan,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Nan,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,Nan,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,Nan,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,Nan,0,0,A.5. 3236,8.05,Nan,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.25,Nan,S
416,1308,3,"Ware, Mr. Frederick",male,Nan,0,0,359309,8.05,Nan,S


In [83]:
out = requests.post('http://172.19.0.1:8089/predict', json={'data':df.to_dict("records")})
out

<Response [422]>

In [59]:
res = requests.post(
    'http://172.19.0.1:8089/predict',
    json = req_data
)
# preds = res.json()
# preds
res

<Response [500]>

In [ ]:
csv_buffer = io.StringIO()
df.to_csv(csv_buffer, index=False)
csv_data = csv_buffer.getvalue()

files = {'file': ('data.csv', csv_data, 'text/csv')}
response = requests.post(
    'http://localhost:8089/predict',
    files=files
)

In [3]:
response

<Response [422]>

In [19]:
df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,None,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,None,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,None,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,None,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,None,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,None,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,None,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,None,S


In [ ]:
data = dataset['data']
    data = pd.DataFrame(data)

    prediction = model.predict(data)
    prediction.to_dict('records')
    return prediction

In [20]:
dat = pd.read_csv("test.csv")
dataset = dat.to_dict(orient='records')

In [23]:
import pickle

with open("pipeline.pkl", "rb") as f:
    model = pickle.load(f)

/usr/local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from version 1.6.1 when using version 1.7.0. This might lead to br

In [24]:
data = pd.DataFrame(dataset)
prediction = model.predict(data)
prediction.to_dict('records')

/usr/local/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[{'ensemble': 0.9791858092654381,
  'RF': 0.9754999999999999,
  'XGB': 0.9830878973007202,
  'LGBM': 0.9789695304955944},
 {'ensemble': 0.8674250366533945,
  'RF': 0.7,
  'XGB': 0.9876204133033752,
  'LGBM': 0.9146546966568083},
 {'ensemble': 0.8027073220810838,
  'RF': 0.86,
  'XGB': 0.635096549987793,
  'LGBM': 0.9130254162554584},
 {'ensemble': 0.4514212848039431,
  'RF': 0.38,
  'XGB': 0.5880892276763916,
  'LGBM': 0.3861746267354377},
 {'ensemble': 0.4884246225262057,
  'RF': 0.47,
  'XGB': 0.3699827194213867,
  'LGBM': 0.6252911481572305},
 {'ensemble': 0.9165167970148261,
  'RF': 0.9225,
  'XGB': 0.9666318297386169,
  'LGBM': 0.8604185613058616},
 {'ensemble': 0.7343025182683841,
  'RF': 0.665,
  'XGB': 0.8858774304389954,
  'LGBM': 0.6520301243661568},
 {'ensemble': 0.9676273372783402,
  'RF': 0.96,
  'XGB': 0.9831923246383667,
  'LGBM': 0.9596896871966539},
 {'ensemble': 0.08687524112156135,
  'RF': 0.14,
  'XGB': 0.04957389831542969,
  'LGBM': 0.07105182504925434},
 {'ensembl